This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

# Best practices for the real world

## Getting the most out of your models

### Hyperparameter optimization

#### Using KerasTuner

In [1]:
!pip install keras-tuner -q

     |████████████████████████████████| 135 kB 21.7 MB/s 


**A KerasTuner model-building function**

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

In [3]:
import keras_tuner as kt

tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=12,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

In [4]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 12 Complete [00h 00m 45s]
val_accuracy: 0.9743500053882599

Best val_accuracy So Far: 0.9753499925136566
Total elapsed time: 00h 11m 02s
INFO:tensorflow:Oracle triggered exit


**Querying the best hyperparameter configurations**

In [13]:
top_n = 1
best_hps = tuner.get_best_hyperparameters(top_n)

In [14]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [21]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model = build_model(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100
391/391 [==============================] - 2s 4ms/step - loss: 0.4640 - accuracy: 0.8763 - val_loss: 0.2398 - val_accuracy: 0.9344
Epoch 2/100
391/391 [==============================] - 1s 3ms/step - loss: 0.2206 - accuracy: 0.9372 - val_loss: 0.1837 - val_accuracy: 0.9514
Epoch 3/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1690 - accuracy: 0.9519 - val_loss: 0.1598 - val_accuracy: 0.9554
Epoch 4/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1402 - accuracy: 0.9599 - val_loss: 0.1387 - val_accuracy: 0.9601
Epoch 5/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1188 - accuracy: 0.9651 - val_loss: 0.1254 - val_accuracy: 0.9623
Epoch 6/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1032 - accuracy: 0.9701 - val_loss: 0.1158 - val_accuracy: 0.9671
Epoch 7/100
391/391 [==============================] - 1s 3ms/step - loss: 0.0904 - accuracy: 0.9736 - val_loss: 0.1096 - val_accuracy: 0.9675

In [24]:
best_models[0].evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0886 - accuracy: 0.9739


[0.08856648206710815, 0.9739000201225281]

In [22]:
best_models = tuner.get_best_models(top_n)

#### The art of crafting the right search space

#### The future of hyperparameter tuning: automated machine learning

### Model ensembling

## Scaling-up model training

### Speeding up training on GPU with mixed precision

#### Understanding floating-point precision

In [ ]:
import tensorflow as tf
import numpy as np
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array)
tf_tensor.dtype

In [ ]:
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array, dtype="float32")
tf_tensor.dtype

#### Mixed-precision training in practice

In [ ]:
from tensorflow import keras
keras.mixed_precision.set_global_policy("mixed_float16")

### Multi-GPU training

#### Getting your hands on two or more GPUs

#### Single-host, multi-device synchronous training

### TPU training

#### Using a TPU via Google Colab

#### Leveraging step fusing to improve TPU utilization

## Summary